# A basic Parfile (Stability test)

## The basic grid
```
Cactus::cctk_run_title = "Maxwell Robust Stability"

Cactus::cctk_full_warnings         = yes
Cactus::highlight_warning_messages = no

# How and when should we end the simulation

Cactus::terminate       = "runtime"
Cactus::max_runtime = 8*60 # 8 hours


#### Setup the grid.              ################################
##################################################################

ActiveThorns = "Carpet CarpetLib CarpetInterp CarpetReduce CarpetSlab"

Carpet::verbose           = no
Carpet::veryverbose       = no
Carpet::schedule_barriers = no
Carpet::storage_verbose   = no
CarpetLib::output_bboxes  = no

Carpet::domain_from_coordbase = yes


Carpet::init_fill_timelevels = yes


#############  Grid setup

ActiveThorns = "CartGrid3D CoordBase"

CartGrid3D::type = "coordbase"
driver::ghost_size       = 3
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "numcells"

# Domain size
CoordBase::xmin = -1.0
CoordBase::ymin = -1.0
CoordBase::zmin = -1.0
CoordBase::xmax = +1.0
CoordBase::ymax = +1.0
CoordBase::zmax = +1.0

# Resolution
CoordBase::ncells_x  = 100
CoordBase::ncells_y  = 100
CoordBase::ncells_z  = 100

CoordBase::boundary_size_x_lower     = 1
CoordBase::boundary_size_y_lower     = 1
CoordBase::boundary_size_z_lower     = 1
CoordBase::boundary_size_x_upper     = 1
CoordBase::boundary_size_y_upper     = 1
CoordBase::boundary_size_z_upper     = 1

CoordBase::boundary_shiftout_x_lower    = 0
CoordBase::boundary_shiftout_y_lower    = 0
CoordBase::boundary_shiftout_z_lower    = 0


### Boundaries
ActiveThorns = "Slab Boundary SymBase"

```
## Time integrator
```
### Time integration. We use RK4
ActiveThorns = "MoL"
mol::ode_method                        = "RK4"
mol::mol_intermediate_steps            = 4
mol::mol_num_scratch_levels            = 1

ActiveThorns = "Time"
Time::dtfac = 0.5  ## Set the Courant factor to 0.5
```
##  The evolution system

```### The actual evolution code
ActiveThorns = "SimpleMaxwell"
ActiveThorns = "Dissipation SphericalSurface SpaceMask"
Dissipation::epsdis                         = 0.2
Dissipation::order   = 5
Dissipation::vars    = "
 SimpleMaxwell::EvolvedVectorD
 SimpleMaxwell::EvolvedVectorB
 SimpleMaxwell::EvolvedScalars
"
```
## Initial Data
```
ActiveThorns = "InitBase "
SimpleMaxwell::material_properties  = "two_blocks"
SimpleMaxwell::initial_data = "random_noise"

SimpleMaxwell::block1_x_start = -.5
SimpleMaxwell::block1_x_end = .5
SimpleMaxwell::block1_y_start = -.5
SimpleMaxwell::block1_y_end = .5
SimpleMaxwell::block1_z_start = -0.75
SimpleMaxwell::block1_z_end =  -0.25

SimpleMaxwell::block2_x_start = -.5
SimpleMaxwell::block2_x_end = .5
SimpleMaxwell::block2_y_start = -.5
SimpleMaxwell::block2_y_end = .5
SimpleMaxwell::block2_z_start = .25
SimpleMaxwell::block2_z_end = .75
SimpleMaxwell::block_transition_width = 0.2


SimpleMaxwell::block1_delta_sigma = 0.5
SimpleMaxwell::block1_delta_epsilon = 0.5
SimpleMaxwell::block1_delta_mu = 0.5
SimpleMaxwell::block2_delta_sigma = 0.5
SimpleMaxwell::block2_delta_epsilon = 0.5
SimpleMaxwell::block2_delta_mu = 0.5


SimpleMaxwell::constant_sigma = 0.0
SimpleMaxwell::constant_epsilon = 1.0
SimpleMaxwell::constant_mu = 1.0
```
## IO and debugging
```
### Debugging checks
ActiveThorns = "NaNChecker"

NaNChecker::check_every     = 16
NaNChecker::action_if_found = "terminate"
NaNChecker::check_vars      = "SimpleMaxwell::EvolvedVectorD SimpleMaxwell::EvolvedVectorB"



### IO
ActiveThorns = "IOUtil"

IO::out_dir = $parfile

### ASCII Output.

### HDF5 Output
ActiveThorns="CarpetIOHDF5"

IO::verbose="full"

ActiveThorns="CarpetIOScalar"
CarpetIOScalar::outScalar_vars = "SimpleMaxwell::EvolvedVectorD"
CarpetIOScalar::outScalar_every = 16
CarpetIOScalar::outScalar_reductions = "norm_inf"

IOHDF5::checkpoint                  = yes
IO::checkpoint_dir                  = "checkpoint"
IO::checkpoint_ID                   = no
IO::checkpoint_every_walltime_hours = 6.0
IO::checkpoint_on_terminate         = yes

IO::recover     = "autoprobe"
IO::recover_dir = "checkpoint"
```


# Parfile with wave and IO

## The grid
```
Cactus::cctk_run_title = "EM wave along z with material"

Cactus::cctk_full_warnings         = yes
Cactus::highlight_warning_messages = no

# How and when should we end the simulation

Cactus::terminate       = "runtime"
Cactus::max_runtime = 60*4


#### Setup the grid. Includes AMR ################################
##################################################################

## AMR
ActiveThorns = "Carpet CarpetLib CarpetInterp CarpetReduce CarpetSlab"

Carpet::verbose           = no
Carpet::veryverbose       = no
Carpet::schedule_barriers = no
Carpet::storage_verbose   = no
#Carpet::timers_verbose    = no
CarpetLib::output_bboxes  = no

Carpet::domain_from_coordbase = yes

Carpet::init_fill_timelevels = yes


#############  Grid setup

ActiveThorns = "CartGrid3D CoordBase"

driver::ghost_size       = 3
CoordBase::domainsize = "minmax"
CoordBase::spacing          = "numcells"

CoordBase::xmin = -5.0
CoordBase::ymin = -5.0
CoordBase::zmin = -5
CoordBase::xmax = +5.0
CoordBase::ymax = +5.0
CoordBase::zmax = +15.0
CoordBase::ncells_x  = 266
CoordBase::ncells_y  = 266
CoordBase::ncells_z  = 532
```
## Boundaries (Periodic except along z)
```
CoordBase::boundary_size_x_lower     = 3
CoordBase::boundary_size_y_lower     = 3
CoordBase::boundary_size_z_lower     = 1
CoordBase::boundary_size_x_upper     = 3
CoordBase::boundary_size_y_upper     = 3
CoordBase::boundary_size_z_upper     = 1

CoordBase::boundary_shiftout_x_lower    = 1
CoordBase::boundary_shiftout_y_lower    = 1
CoordBase::boundary_shiftout_z_lower    = 0


### Boundaries
ActiveThorns = "Periodic Slab Boundary SymBase"
CartGrid3D::type = "coordbase"
Periodic::periodic_x = "yes"
Periodic::periodic_y = "yes"
Periodic::periodic_z = "no"
Periodic::verbose    = "no"
```
##  The evolution system

```### The actual evolution code
ActiveThorns = "SimpleMaxwell"
ActiveThorns = "Dissipation SphericalSurface SpaceMask"
Dissipation::epsdis                         = 0.2
Dissipation::order   = 5
Dissipation::vars    = "
 SimpleMaxwell::EvolvedVectorD
 SimpleMaxwell::EvolvedVectorB
 SimpleMaxwell::EvolvedScalars
"
```
## Time integrator
```
### Time integration. We use RK4
ActiveThorns = "MoL"
mol::ode_method                        = "RK4"
mol::mol_intermediate_steps            = 4
mol::mol_num_scratch_levels            = 1

ActiveThorns = "Time"
Time::dtfac = 0.5  ## Set the Courant factor to 0.5
```
## Initial data

```
ActiveThorns = "InitBase "

SimpleMaxwell::material_properties  = "two_blocks"

SimpleMaxwell::block1_z_start = 3.0
SimpleMaxwell::block1_z_end = 4.0
SimpleMaxwell::block1_x_start = -10
SimpleMaxwell::block1_x_end = -.5

SimpleMaxwell::block2_z_start = 3
SimpleMaxwell::block2_z_end = 4
SimpleMaxwell::block2_x_start = .5
SimpleMaxwell::block2_x_end = 10
SimpleMaxwell::block_transition_width = 0.25


SimpleMaxwell::block1_delta_sigma = 1.0
SimpleMaxwell::block1_delta_epsilon = 0.0
SimpleMaxwell::block1_delta_mu = 0.0
SimpleMaxwell::block2_delta_sigma = 1.0
SimpleMaxwell::block2_delta_epsilon = 0.0
SimpleMaxwell::block2_delta_mu = 0.0


SimpleMaxwell::constant_sigma = 0.0
SimpleMaxwell::constant_epsilon = 1.0
SimpleMaxwell::constant_mu = 1.0
SimpleMaxwell::initial_data = "compact_pulse_plus_z"
```
## IO
```

### Debugging checks
ActiveThorns = "NaNChecker"

NaNChecker::check_every     = 16
NaNChecker::action_if_found = "terminate"
NaNChecker::check_vars      = "SimpleMaxwell::EvolvedVectorD SimpleMaxwell::EvolvedVectorB"



### IO
ActiveThorns = "IOUtil"

IO::out_dir = $parfile

### ASCII Output.
ActiveThorns=  "CarpetIOASCII"
CarpetIOASCII::out1D_every                    = 2
#

### HDF5 Output
ActiveThorns="CarpetIOHDF5"

IO::verbose="full"
IOHDF5::out2D_every            = 16
IOHDF5::out2D_vars            = "SimpleMaxwell::EvolvedVectorD SimpleMaxwell::EvolvedVectorB SimpleMaxwell::EvolvedScalars  SimpleMaxwell::EvolvedConstraintScalars SimpleMaxwell::StaticFields"
IOHDF5::out3D_every            = 2048
IOHDF5::out3D_vars            = "SimpleMaxwell::EvolvedVectorD SimpleMaxwell::EvolvedVectorB SimpleMaxwell::EvolvedScalars SimpleMaxwell::EvolvedConstraintScalars SimpleMaxwell::StaticFields"
IOHDF5::one_file_per_group     = yes
IOHDF5::output_symmetry_points = no
IOHDF5::compression_level      = 1
IOHDF5::use_checksums          = yes
ActiveThorns="CarpetIOScalar"
CarpetIOScalar::outScalar_vars = "SimpleMaxwell::EvolvedVectorD"
CarpetIOScalar::outScalar_every = 16
CarpetIOScalar::outScalar_reductions = "norm_inf"

IOHDF5::checkpoint                  = yes
IO::checkpoint_dir                  = "checkpoint"
IO::checkpoint_ID                   = no
IO::checkpoint_every_walltime_hours = 6.0
IO::checkpoint_on_terminate         = yes

IO::recover     = "autoprobe"
IO::recover_dir = "checkpoint
```